In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torch.utils.data import random_split
from PIL import Image 
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset

class AnimalDataset(Dataset):
    def __init__(self, root_dir, transform=None,augmentation_fn=None):
        """
        Args:
          root_dir (str): Directory with subfolders per class.
          transform (callable, optional): Transform to apply to PIL images.
        """
        self.root_dir = root_dir
        self.transform = transform

        # Build list of (image_path, label) pairs
        self.samples = []
        self.class_to_idx = {}
        for idx, class_name in enumerate(sorted(os.listdir(root_dir))):
            class_folder = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_folder):
                continue
            self.class_to_idx[class_name] = idx
            for fname in os.listdir(class_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(class_folder, fname)
                    self.samples.append((path, idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path, label = self.samples[index]
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [3]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

import random

def my_fourier_augmentation(image):
    transforms = [
        GaussianMixture,
        AmplitudeRescale,
        RandomFrequencyMask,
        PhaseShift
    ]
    lmage = random.choice(transforms)(image)
    return image

# 1) Transforms: resize, to tensor, normalize (ImageNet stats)
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225])
])

# 2) Instantiate full dataset
full_dataset = AnimalDataset(root_dir='/kaggle/input/animals10/raw-img', transform=data_transform, augmentation_fn=my_fourier_augmentation)


# 3) Split into train (80%) and test (20%)
train_size = int(0.8 * len(full_dataset))
test_size  = len(full_dataset) - train_size
train_ds, test_ds = random_split(full_dataset, [train_size, test_size])
# Since PyTorch v1.13, you can also pass floats: [0.8, 0.2] :contentReference[oaicite:6]{index=6}

# 4) DataLoaders for batching
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=4)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=4)


In [4]:
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained MobileNetV3-Large and replace final classification layer
model = models.mobilenet_v3_large(pretrained=True)
num_classes = len(full_dataset.class_to_idx)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[3].parameters(), lr=1e-3)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 133MB/s] 


In [5]:
from tqdm import tqdm

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss, running_corrects = 0.0, 0

    # Wrap the training loader with tqdm
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        running_corrects += (preds == labels).sum().item()

    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects / train_size * 100
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%")

    # Evaluation
    model.eval()
    test_corrects, test_total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            test_corrects += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = test_corrects / test_total * 100
    print(f"          Test Acc: {test_acc:.2f}%\n")


Epoch 1/10 - Train Loss: 0.5271, Train Acc: 83.36%


          Test Acc: 86.54%



Epoch 2/10 - Train Loss: 0.4059, Train Acc: 86.27%


          Test Acc: 86.59%



Epoch 3/10 - Train Loss: 0.3924, Train Acc: 86.43%


...

          Test Acc: 86.88%



Epoch 9/10 - Train Loss: 0.3632, Train Acc: 87.17%


          Test Acc: 88.39%



Epoch 10/10 - Train Loss: 0.3638, Train Acc: 87.39%


          Test Acc: 87.85%

